# **CNN 入門**
此份程式碼會介紹透過一個簡單的公開資料集，建置模型、訓練模型，並比較 DNN model 處理影像型資料的差異。

## 匯入所需套件

In [ ]:
# import package
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import (Input, Dense, Dropout, Activation,
                                     BatchNormalization, Flatten,
                                     Conv2D, MaxPooling2D)

## Cifar10 資料讀入及前處理

![](https://i.imgur.com/edFmvOC.png)

In [ ]:
# cifar10 中有將 data 先分為 train 和 test
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

print('x_train shape:', x_train.shape)
print('y_train.shape:', y_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# x_train.shape: 四個維度：第 1 維度為筆數、第 2, 3 維度為影像大小 32*32、第 4 維度是 RGB 三原色，所以是 3
# x_train 中有 50000 筆訓練資料，以及 x_test 中有 10000 筆的測試資料

In [ ]:
uniques, counts = np.unique(y_train, return_counts=True)
print(uniques, counts)

plt.bar(uniques, counts)
plt.xticks(uniques)
plt.show()

In [ ]:
uniques, counts = np.unique(y_test, return_counts=True)
print(uniques, counts)

plt.bar(uniques, counts)
plt.xticks(uniques)
plt.show()

In [ ]:
plt.imshow(x_train[0])
plt.title("label: {}".format(y_train[0]), fontsize=15)     # 第 0 筆圖像資料分類的位置
plt.axis("off")
plt.show()

In [ ]:
x_train.dtype

In [ ]:
# 記得轉成 'float32'
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

# 將 features (照片影像特徵值) 標準化，可以提高模型預測的準確度，並且更快收斂
x_train /= 255  # rescaling
x_test /= 255   # rescaling

In [ ]:
y_train[0:10]    # 0:10筆的類別

In [ ]:
# 將訓練資料與測試資料的 label，進行 Onehot encoding 轉換
num_classes = 10
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

# y_train = np.eye(num_classes, dtype='float32')[y_train[:, 0]]
# y_test = np.eye(num_classes, dtype='float32')[y_test[:, 0]]

print('y_train shape:', y_train.shape)
print('y_test shape:', y_test.shape)

## 模型定義

In [ ]:
dnn_model = Sequential()

dnn_model.add(Flatten(input_shape=x_train.shape[1:]))
dnn_model.add(Dense(32))
dnn_model.add(Activation('relu'))
dnn_model.add(Dense(32))
dnn_model.add(Activation('relu'))
dnn_model.add(Dense(64))
dnn_model.add(Activation('relu'))
dnn_model.add(Dense(64))
dnn_model.add(Activation('relu'))
dnn_model.add(Dense(num_classes))
dnn_model.add(Activation('softmax'))

In [ ]:
dnn_model.summary()

* ### CNN Model
![](https://i.imgur.com/fyoPGuk.png)

In [ ]:
# 選擇 Keras 的 API 寫法
inputs = Input(shape=x_train.shape[1:])

# 第一層
# 建立卷積層，設定32個3*3的filters
# 設定ReLU為激活函數。
x = Conv2D(32, (3, 3), activation='relu')(inputs)

# 第二層 - 卷積層 + 池化層
x = Conv2D(32, (3, 3), activation='relu')(x)
x = MaxPooling2D(pool_size=(2, 2))(x)

# 第三層 - 卷積層
x = Conv2D(64, (3, 3), activation='relu')(x)

# 第四層 - 卷積層 + 池化層
x = Conv2D(64, (3, 3), activation='relu')(x)
x = MaxPooling2D(pool_size=(2, 2))(x)

# 建立分類模型 (MLP) : 平坦層 + 輸出層 (10)
x = Flatten()(x)
outputs = Dense(num_classes, activation='softmax')(x)


cnn_model = Model(inputs=inputs, outputs=outputs)

In [ ]:
cnn_model.summary()

In [ ]:
# 編譯模型
# 選用 Adam 為 optimizer
learning_rate = 0.0001
optimizer = keras.optimizers.Adam(learning_rate=learning_rate)

dnn_model.compile(loss='categorical_crossentropy',
                  optimizer=optimizer,
                  metrics=['accuracy'])
cnn_model.compile(loss='categorical_crossentropy',
                  optimizer=optimizer,
                  metrics=['accuracy'])

## 開始訓練模型

In [ ]:
batch_size = 32
epochs = 20

In [ ]:
print('Training DNN model')
dnn_history = dnn_model.fit(x_train, y_train,
                            batch_size=batch_size,
                            epochs=epochs,
                            validation_data=(x_test, y_test),
                            verbose=1)
print('Training CNN model')
cnn_history = cnn_model.fit(x_train, y_train,
                            batch_size=batch_size,
                            epochs=epochs,
                            validation_data=(x_test, y_test),
                            verbose=1)

## 測試資料

In [ ]:
test_pred = cnn_model.predict(x_test[0:1]).argmax(-1)

plt.imshow(x_test[0])
print('prediction: ', test_pred)

In [ ]:
print(y_test[0])

In [ ]:
dnn_loss, dnn_acc = dnn_model.evaluate(x_test, y_test, verbose=2)
cnn_loss, cnn_acc = cnn_model.evaluate(x_test, y_test, verbose=2)

## 訓練結果視覺化

In [ ]:
history_list = [cnn_history, dnn_history]
history_train_acc = ["cnn_train_acc", "dnn_train_acc"]
history_valid_acc = ["cnn_valid_acc", "dnn_valid_acc"]
history_train_loss = ["cnn_train_loss", "dnn_train_loss"]
history_valid_loss = ["cnn_valid_loss", "dnn_valid_loss"]

In [ ]:
plt.figure(figsize=(20, 6))

# training loss
plt.subplot(1, 2, 1)
for each_his, each_train, each_valid in zip(history_list,
                                            history_train_loss,
                                            history_valid_loss):
    l_x = len(each_his.history['loss'])
    plt.plot(np.arange(l_x), each_his.history['loss'], label=each_train)
    plt.plot(np.arange(l_x), each_his.history['val_loss'], label=each_valid)
plt.legend(loc='best')
plt.title('Loss')

# training acc
plt.subplot(1, 2, 2)
for each_his, each_train, each_valid in zip(history_list,
                                            history_train_acc,
                                            history_valid_acc):
    l_x = len(each_his.history['accuracy'])
    plt.plot(np.arange(l_x), each_his.history['accuracy'], label=each_train)
    plt.plot(np.arange(l_x), each_his.history['val_accuracy'], label=each_valid)
plt.legend(loc='best')
plt.title('Accuracy')
plt.show()